In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib
from fastapi import FastAPI, File, UploadFile
from io import BytesIO
from pyngrok import ngrok

In [7]:
file_path = 'E:\Мои документы\Desktop\Уроки\Python\Laptop_price.csv'
df = pd.read_csv(file_path)

In [8]:
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
pipeline = Pipeline([
    ('preprocessor', preprocessor),
   ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, 'E:\Мои документы\Desktop\Уроки\Python\laptop_price_model.pkl')

['E:\\Мои документы\\Desktop\\Уроки\\Python\\laptop_price_model.pkl']

In [10]:
app = FastAPI()

# Загрузка обученной модели
model_path = "E:\Мои документы\Desktop\Уроки\Python\laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}

In [11]:
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)


t=2025-03-15T18:14:05+0300 lvl=eror msg="unable to evaluate ngrok agent binary path for symlinks" obj=tunnels.session err="CreateFile C:\\Users\\Yura\\AppData\\Local\\ngrok\\ngrok.exe: The system cannot find the file specified."


API доступно по адресу: NgrokTunnel: "https://73f8-185-42-163-191.ngrok-free.app" -> "http://localhost:8000"


t=2025-03-15T18:14:10+0300 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"
